In [1]:
import numpy as np
from load import load_data
from convert import convert
from pmc2 import *


In [2]:
#%matplotlib widget
%matplotlib inline
#%matplotlib qt
#%matplotlib gtk

import matplotlib.pyplot as plt
import matplotlib.font_manager

from mpl_toolkits.mplot3d import Axes3D
from mpl_toolkits.mplot3d.art3d import Poly3DCollection

plt.rcParams.update({
    'pgf.rcfonts': False,
})

plt.rcParams['font.family'] = 'serif'

#from matplotlib.backends.backend_pgf import FigureCanvasPgf
#matplotlib.backend_bases.register_backend('pdf', FigureCanvasPgf)

In [3]:
alf_inter = 1
alf_plane = 0.5

# Input

- bpC : CT Compression data Break Point in p-q plane 
- bpE : CT Extension data Break Point in p-q plane

In [4]:
bpC = 150 #CT Compression data Break Point in p-q plane
bpE = 158 #CT Extension data Break Point in p-q plane 
bpo = 2


In [5]:
data = load_data(10)
d = convert(data)
P1_init, P2_init = create_P1_and_P2(data,bpC,bpE,bpo)
#print(P1.sol)
#print(P2.sol)
if P1_init.Vo < P2_init.Vo :
    P1 = P2_init
    P2 = P1_init
elif P1_init.Vo > P2_init.Vo :
    P1 = P1_init
    P2 = P2_init
print(P1.sol)
print(P2.sol)

NameError: name 'create_P1_and_P2' is not defined

In [6]:
fig = plt.figure()
ax = fig.add_subplot(111, projection='3d')
#ax.plot(d['sig2'],d['sig3'],d['sig1'],'k.')
ax.plot(*P1.pts,'k.')
ax.plot(*P2.pts,'k.')
#ax.plot(d['sig2'],d['sig3'],d['sig1'],'k.')

In [7]:
#print('P1 P2\n{} {}\n{}{}'.format(P1['phyC'],P2['phyC'],P1['phyE'],P2['phyE']))

In [8]:
eq_pts_transisiton_P12 = np.zeros((6,3,3))

for i in range(6):
    eq_pts_transisiton_P12[i] = np.array([get_plane_normal_6_cycle(P1,i),get_plane_normal_6_cycle(P1,i-1),get_plane_normal_6_cycle(P2,i)])

ones = np.ones((6,3))

pts_transisiton_P12 = np.linalg.solve(eq_pts_transisiton_P12, ones).transpose()

In [9]:
eq_pts_P2 = np.array([get_plane_normal_6_cycle(P2,0),get_plane_normal_6_cycle(P2,1),get_plane_normal_6_cycle(P2,2)])
pts_P2 = np.linalg.solve(eq_pts_P2,np.ones((3,1)))

In [10]:
fig.show()

In [11]:
sig_pts = np.concatenate((P1.pts,P2.pts),axis=1) #miss pts_P2
pts_with_largest_coord = sig_pts[:,np.argmax(np.max(sig_pts, axis=0))]
dist = np.dot(np.ones(3)/np.sqrt(3),pts_with_largest_coord)*1.5

In [12]:
pts_P1_far = p_plane_intersection_6(P1,dist)

In [13]:
ax.plot(*pts_P2,'ko',alpha=0)
ax.plot(*pts_transisiton_P12,'ro',alpha=0)
ax.plot(*pts_P1_far,'bo',alpha=0)
#alpha=alf_inter

In [14]:
def get_verts(coord):
    return [list(zip(*cord))]

## Draw P2 planes

In [15]:
for i in range(6):
    cord = np.zeros((3,3))
    cord[:,0] = pts_P2[:,0]
    cord[:,1] = pts_transisiton_P12[:,i]
    cord[:,2] = pts_transisiton_P12[:,(i+1)%6]
    
    verts = get_verts(cord)
    poly = Poly3DCollection(verts,linewidths=1, alpha=alf_plane)
    poly.set_facecolor('r')
    ax.add_collection3d(poly)

## Draw P1 planes

In [16]:
for i in range(6):
    cord = np.zeros((3,4))
    cord[:,0] = pts_transisiton_P12[:,i]
    cord[:,1] = pts_transisiton_P12[:,(i+1)%6]
    cord[:,2] = pts_P1_far[:,(i+1)%6]
    cord[:,3] = pts_P1_far[:,i%6]
    
    verts = get_verts(cord)
    poly = Poly3DCollection(verts,linewidths=1, alpha=0.5)
    poly.set_facecolor('b')
    ax.add_collection3d(poly)

In [17]:
fig.show()
#fig.savefig('3D.pdf')

# 2D view

In [17]:
new_base = np.zeros((3,3))

new_z = np.array([1,1,1])
new_y = np.array([-1,-1,2])
new_x = np.array([-1,1,0])
new_base[:,0] = new_x /np.linalg.norm(new_x )
new_base[:,1] = new_y /np.linalg.norm(new_y )
new_base[:,2] = new_z /np.linalg.norm(new_z )
new_base_transform = np.linalg.inv(new_base)

In [18]:
new_sig = new_base_transform @ sig_pts
new_transi = new_base_transform @ pts_transisiton_P12
new_P2 = new_base_transform @ pts_P2
new_far_P1 = new_base_transform @ pts_P1_far

In [19]:
transi_dist_min = np.min(new_transi[2,:])
near_inter = p_plane_intersection_6(P2,transi_dist_min*0.99)
new_near = new_base_transform @ near_inter

In [20]:
transi_dist_max = np.max(new_transi[2,:])
t = 0.3
dist = transi_dist_max*t + transi_dist_min*(1-t)

offset = 0 if np.allclose(new_transi[0,0],transi_dist_max) else 2
middle_inter = p_plane_intersection_12(P1,P2,dist,offset)
new_middle = new_base_transform @ middle_inter

In [21]:
plt.close()
#plt.axis('equal')
plt.plot(new_sig[0,:],new_sig[1,:],'k.')
plt.plot(new_P2[0,:],new_P2[1,:],'r.')
#plt.fill(new_P12[0,:],new_P12[1,:],edgecolor='r',fill=False)
plt.fill(new_far_P1[0,:],new_far_P1[1,:],edgecolor='b',fill=False)
plt.fill(new_near[0,:],new_near[1,:],edgecolor='r',fill=False)
plt.fill(new_middle[0,:],new_middle[1,:],edgecolor='y',fill=False)

In [25]:
x = np.linspace(0,700)
zer = [0]*len(x)
v = np.array([x,x,zer])
w = new_base_transform@v
plt.plot(w[0,:],w[1,:],'k--')
v = np.array([x,zer,x])
w = new_base_transform@v
plt.plot(w[0,:],w[1,:],'k--')
v = np.array([zer,x,x])
w = new_base_transform@v
plt.plot(w[0,:],w[1,:],'k--')

x = np.linspace(-700,0)
zer = [0]*len(x)
v = np.array([x,x,zer])
w = new_base_transform@v
plt.plot(w[0,:],w[1,:],'k-')
v = np.array([x,zer,x])
w = new_base_transform@v
plt.plot(w[0,:],w[1,:],'k-')
v = np.array([zer,x,x])
w = new_base_transform@v
plt.plot(w[0,:],w[1,:],'k-')

In [26]:
plt.axis('equal')
plt.show()

In [27]:
plt.close()

In [28]:
P1.sig123

array([[ 71.5,  98.4,  95.9, 114.5, 110.9, 129.4, 125.5, 142.1, 138.1,
        153.8, 150.8,  50. ,  50. ,  60. ,  60. ,  69. ,  90. , 120. ,
        134. ,  48.3,  77.5,  83.5],
       [ 10. ,  20. ,  20. ,  30. ,  30. ,  40. ,  40. ,  50. ,  50. ,
         60. ,  60. ,  50. ,  50. ,  60. ,  60. ,  69. ,  90. , 120. ,
        134. ,  31.6,  70.5,  60.5],
       [ 10. ,  20. ,  20. ,  30. ,  30. ,  40. ,  40. ,  50. ,  50. ,
         60. ,  60. ,   6. ,   5.7,  10.1,   8. ,  11.5,  25.7,  49.3,
         55.7,   5. ,  20. ,  24. ]])

In [29]:
P2.sig123

array([[29.7, 24.9, 39.4, 35.2, 52.9, 48.8, 68. , 35. , 40. , 40. , 53. ,
        63.9],
       [ 0. ,  0. ,  2.5,  2.5,  5. ,  5. , 10. , 35. , 40. , 40. , 25.1,
        12.1],
       [ 0. ,  0. ,  2.5,  2.5,  5. ,  5. , 10. ,  0.8,  1.2,  1.8,  7. ,
         9. ]])

In [28]:
d['pC']

array([ 33.3,  74.8, 106.1, 130.4, 153.3, 172.9, 211.8])

In [29]:
d['pE']

array([], dtype=float64)

In [30]:
d['po']

array([138.6, 146.1, 158.3, 177.3, 203.2, 231.6, 240.4, 165.4, 192.7,
       215.8, 225.3, 239.4, 249.8, 195.6, 212.7, 240.9, 264.5, 274.7,
       297.5, 229. , 254.4, 284.4, 325.8, 332.5])

In [31]:
P1.p

array([203.2, 231.6, 240.4, 225.3, 239.4, 249.8, 264.5, 274.7, 297.5,
       284.4, 325.8, 332.5])

In [32]:
P2.p

array([ 33.3,  74.8, 106.1, 130.4, 153.3, 172.9, 211.8, 138.6, 146.1,
       158.3, 177.3, 165.4, 192.7, 215.8, 195.6, 212.7, 240.9, 229. ,
       254.4])

In [33]:
P1.sol

array([141.12750166, 104.71519753,  -0.97887085, 126.77513864,
        28.00167102,  28.61537667])

In [34]:
P2.sol

array([86.3897576 , 78.48390736, -0.6936657 , 69.38426942, 31.03484639,
       44.18443903])